# 1

In [21]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Example") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3631 B]
Hit:3 https://dl.yarnpkg.com/debian stable InRelease                           
Get:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3960 B]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease   
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [258 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1174 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3330 kB]33m
Hit:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease              
Get:12 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable/main amd64 Packages [3828 B]m
Hit:9 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease            


24/02/05 19:18:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [26]:
import pandas as pd
df = pd.read_json("https://data.cityofnewyork.us/resource/uip8-fykc.json")
spark_df = spark.createDataFrame(df)

df.head(5)

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,x_coord_cd,y_coord_cd,latitude,longitude,geocoded_column,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_efsh_h5xi
0,261265483,2023-01-03T00:00:00.000,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1600500,F,B,49,...,1027430,251104,40.855793,-73.843908,"{'type': 'Point', 'coordinates': [-73.843908, ...",59,5,12,32,11270.0
1,261271301,2023-01-03T00:00:00.000,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,S,120,...,962808,174275,40.644996,-74.077263,"{'type': 'Point', 'coordinates': [-74.077263, ...",4,1,13,74,10369.0
2,261336449,2023-01-04T00:00:00.000,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 1601001,F,K,61,...,995118,155708,40.594054,-73.960866,"{'type': 'Point', 'coordinates': [-73.960866, ...",32,2,15,36,18183.0
3,261328047,2023-01-04T00:00:00.000,105,STRANGULATION 1ST,106,FELONY ASSAULT,PL 1211200,F,Q,114,...,1007694,219656,40.769552,-73.915361,"{'type': 'Point', 'coordinates': [-73.915361, ...",39,3,4,72,16860.0
4,261417496,2023-01-05T00:00:00.000,244,"BURGLARY,UNCLASSIFIED,UNKNOWN",107,BURGLARY,PL 1402000,F,B,44,...,1007174,239542,40.824135,-73.917170,"{'type': 'Point', 'coordinates': [-73.91717, 4...",34,5,43,25,10929.0


In [31]:
import urllib.request

# Download the JSON file locally
json_url = "https://data.cityofnewyork.us/resource/uip8-fykc.json"
local_file_path = "/workspaces/Procesamiento-Datos/Project/Data/data.json"
urllib.request.urlretrieve(json_url, local_file_path)

# Read local JSON file into Spark DataFrame
spark_df = spark.read.json(local_file_path)

# Show the first 5 rows
spark_df.show(5)


AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [32]:
# Show the first 5 rows without considering corrupt records
spark_df.drop("_corrupt_record").show(5)


++
||
++
||
||
||
||
||
++
only showing top 5 rows



In [27]:
spark_df.show(5)

+----------+--------------------+-----+--------------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+---------+----------+--------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+
|arrest_key|         arrest_date|pd_cd|             pd_desc|ky_cd|     ofns_desc|  law_code|law_cat_cd|arrest_boro|arrest_precinct|jurisdiction_code|age_group|perp_sex|perp_race|x_coord_cd|y_coord_cd| latitude| longitude|     geocoded_column|:@computed_region_f5dn_yrer|:@computed_region_yeji_bk3q|:@computed_region_92fq_4b7q|:@computed_region_sbqj_enih|:@computed_region_efsh_h5xi|
+----------+--------------------+-----+--------------------+-----+--------------+----------+----------+-----------+---------------+-----------------+---------+--------+---------+----------+----------+---------+----------+-------------

# Bonus

## Web Scrapping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website
url = "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoodpop.htm"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with the specified class
table = soup.find('table', {'class': 'light_table right'})

# Extract the table rows
rows = table.find_all('tr')

# Extracting data from each row
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Create a DataFrame from the extracted data
columns = data[0]  # Assuming the first row contains column headers
df = pd.DataFrame(data[1:], columns=columns)

# Display the head of the DataFrame
print(df.head())


  Borough                                     region   Males Females  \
0   Bronx         Riverdale, Fieldston & Kingsbridge  52,133  61,937   
1               Wakefield, Williamsbridge & Woodlawn  65,087  77,848   
2             Co-op City, Pelham Bay & Schuylerville  55,615  65,929   
3              Pelham Parkway, Morris Park & Laconia  61,233  67,896   
4          Belmont, Crotona Park East & East Tremont  75,963  87,740   

  Total Population  
0          114,070  
1          142,935  
2          121,544  
3          129,130  
4          163,704  


## API

In [5]:
import requests

# Replace 'YOUR_API_KEY' with your actual OpenWeatherMap API key
api_key = '69f9153f8e4f7e6c52715b0b75ababe3'
latitude = '4.7110'  # Replace with the desired latitude
longitude = '74.072'  # Replace with the desired longitude

# Make a request to the One Call API
url = f'https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}'
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract the relevant information from the JSON response
    current_temperature = data['current']['temp']
    current_weather_description = data['current']['weather'][0]['description']

    # Print the extracted information
    print(f'Current Temperature: {current_temperature} K')
    print(f'Current Weather: {current_weather_description}')
else:
    # Print an error message if the request was not successful
    print(f'Error: {response.status_code}')


Error: 401
